Generate, load and prepare policy histories

In [1]:
import os
from datetime import datetime
import support_functions as sf


PROJECT_ROOT_DIR = "/Users/gopora/MyStuff/Dev/Workspaces/Sandbox/TroubledLife"
DATASETS_DIR = os.path.join(PROJECT_ROOT_DIR, "data")
TF_LOG_DIR = os.path.join(PROJECT_ROOT_DIR, "tf_logs")
TRAINING_SET_DATA_FILE = "troubled_life_policy_train_data.csv"
TEST_SET_DATA_FILE = "troubled_life_policy_test_data.csv"

now = datetime.utcnow().strftime("%Y%m%d-%H%M%S")
log_dir = "{}/run-{}/".format(TF_LOG_DIR, now)

sf.generate_troubled_life_policy_data(no_of_policies=10000, runtime=5, file_path=os.path.join(DATASETS_DIR, TRAINING_SET_DATA_FILE))

policy_histories_train = \
    sf.load_troubled_life_policy_data(file_path=os.path.join(DATASETS_DIR, TRAINING_SET_DATA_FILE))
policy_histories_length_train, max_policy_history_length_train = \
    sf.get_policy_history_lengths(policy_histories=policy_histories_train)

sf.generate_troubled_life_policy_data(no_of_policies=2000, runtime=5, file_path=os.path.join(DATASETS_DIR, TEST_SET_DATA_FILE))

policy_histories_test = \
    sf.load_troubled_life_policy_data(file_path=os.path.join(DATASETS_DIR, TEST_SET_DATA_FILE))
policy_histories_length_test, max_policy_history_length_test = \
    sf.get_policy_history_lengths(policy_histories=policy_histories_test)

# Pad the histories up to maximum length of both, train and test set
max_policy_history_length = max(max_policy_history_length_train, max_policy_history_length_test)
policy_histories_train = \
    sf.pad_troubled_life_policy_histories(policy_histories=policy_histories_train,
                                          policy_histories_lengths=policy_histories_length_train,
                                          max_policy_history_length=max_policy_history_length)
policy_histories_test = \
    sf.pad_troubled_life_policy_histories(policy_histories=policy_histories_test,
                                          policy_histories_lengths=policy_histories_length_test,
                                          max_policy_history_length=max_policy_history_length)

# Save padded data, since always generating and padding takes too long
policy_histories_train.to_csv(path_or_buf=os.path.join(DATASETS_DIR, TRAINING_SET_DATA_FILE))
policy_histories_test.to_csv(path_or_buf=os.path.join(DATASETS_DIR, TEST_SET_DATA_FILE))

Prepare labels and features

In [2]:
# Extract features and labels from dataset as numpy.ndarray(s)
train_labels, train_features, train_seq_lengths =\
    sf.prepare_labels_features_lengths(policy_histories=policy_histories_train,
                                       policy_histories_lengths=policy_histories_length_train, 
                                       max_policy_history_length=max_policy_history_length)
test_labels, test_features, test_seq_lengths =\
    sf.prepare_labels_features_lengths(policy_histories=policy_histories_test,
                                       policy_histories_lengths=policy_histories_length_test, 
                                       max_policy_history_length=max_policy_history_length)

Build TensorFlow graph with RNN

In [3]:
import tensorflow as tf

tf.reset_default_graph()
tf.set_random_seed(42)

# Our hyperparameters
n_steps = max_policy_history_length
n_inputs = 2
n_neurons = 20
n_outputs = 2
learning_rate = 0.001

# Placeholders for our input sequences
seq_length = tf.placeholder(tf.int32, [None], name="seq_length")
X = tf.placeholder(tf.float32, [None, n_steps, n_inputs], name="X")
y = tf.placeholder(tf.int32, [None], name="y")

# A single RNN layer of n_neurons neurons
basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons, activation=tf.nn.relu)
outputs, states = tf.nn.dynamic_rnn(basic_cell, X, dtype=tf.float32, sequence_length=seq_length)

# We feed the layer's output of the last timestep into a fully connected layer of 2 neurons
# (1 per class, troubled or not). Softmax layer is next
logits = tf.layers.dense(states, n_outputs)
y_pred = tf.argmax(tf.nn.softmax(logits), axis=1)
xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)

# Loss function and Adam Optimizer
loss = tf.reduce_mean(tf.cast(xentropy, tf.float32))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(loss)

correct = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.global_variables_initializer()

accuracy_summary = tf.summary.scalar('Accuracy', accuracy)
loss_summary = tf.summary.scalar('Loss', loss)

file_writer = tf.summary.FileWriter(log_dir, tf.get_default_graph())

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

n_epochs = 100
batch_size = 200

train_data = sf.TrainDataSet(train_labels=train_labels, train_features=train_features, train_seq_lengths=train_seq_lengths)

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for i in range(train_data.num_examples // batch_size):
            y_batch, X_batch, seq_length_batch = train_data.next_batch(batch_size)
            
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch, seq_length: seq_length_batch})

        y_train_pred = y_pred.eval(feed_dict={X: X_batch, y: y_batch, seq_length: seq_length_batch})
        y_test_pred = y_pred.eval(feed_dict={X: test_features, y: test_labels, seq_length: test_seq_lengths})
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch, seq_length: seq_length_batch})
        acc_test = accuracy.eval(feed_dict={X: test_features, y: test_labels, seq_length: test_seq_lengths})

        print(epoch, "Train accuracy:", acc_train, "Test accuracy:", acc_test)
        print(confusion_matrix(y_batch, y_train_pred))
        print(classification_report(y_batch, y_train_pred, target_names=['ok', 'troubled']))

        loss_summary_str = loss_summary.eval(feed_dict={X: X_batch, y: y_batch, seq_length: seq_length_batch})
        accuracy_summary_str = accuracy_summary.eval(feed_dict={X: X_batch, y: y_batch, seq_length: seq_length_batch})
        file_writer.add_summary(loss_summary_str, epoch)
        file_writer.add_summary(accuracy_summary_str, epoch)

file_writer.close()

0 Train accuracy: 0.2 Test accuracy: 0.177
[[  0 148]
 [ 12  40]]
             precision    recall  f1-score   support

         ok       0.00      0.00      0.00       148
   troubled       0.21      0.77      0.33        52

avg / total       0.06      0.20      0.09       200

1

 Train accuracy: 0.88 Test accuracy: 0.874
[[158   0]
 [ 24  18]]
             precision    recall  f1-score   support

         ok       0.87      1.00      0.93       158
   troubled       1.00      0.43      0.60        42

avg / total       0.90      0.88      0.86       200



2 Train accuracy: 0.88 Test accuracy: 0.894
[[155   0]
 [ 24  21]]
             precision    recall  f1-score   support

         ok       0.87      1.00      0.93       155
   troubled       1.00      0.47      0.64        45

avg / total       0.90      0.88      0.86       200



3 Train accuracy: 0.855 Test accuracy: 0.8895
[[151   0]
 [ 29  20]]
             precision    recall  f1-score   support

         ok       0.84      1.00      0.91       151
   troubled       1.00      0.41      0.58        49

avg / total       0.88      0.85      0.83       200

4 Train accuracy: 0.92 Test accuracy: 0.9105
[[162   0]
 [ 16  22]]
             precision    recall  f1-score   support

         ok       0.91      1.00      0.95       162
   troubled       1.00      0.58      0.73        38

avg / total       0.93      0.92      0.91       200



5 Train accuracy: 0.925 Test accuracy: 0.9215
[[155   0]
 [ 15  30]]
             precision    recall  f1-score   support

         ok       0.91      1.00      0.95       155
   troubled       1.00      0.67      0.80        45

avg / total       0.93      0.93      0.92       200

6 Train accuracy: 0.87 Test accuracy: 0.9005
[[152   0]
 [ 26  22]]
             precision    recall  f1-score   support

         ok       0.85      1.00      0.92       152
   troubled       1.00      0.46      0.63        48

avg / total       0.89      0.87      0.85       200



...

91 Train accuracy: 1.0 Test accuracy: 1.0
[[153   0]
 [  0  47]]
             precision    recall  f1-score   support

         ok       1.00      1.00      1.00       153
   troubled       1.00      1.00      1.00        47

avg / total       1.00      1.00      1.00       200

92 Train accuracy: 1.0 Test accuracy: 1.0
[[157   0]
 [  0  43]]
             precision    recall  f1-score   support

         ok       1.00      1.00      1.00       157
   troubled       1.00      1.00      1.00        43

avg / total       1.00      1.00      1.00       200



93 Train accuracy: 1.0 Test accuracy: 1.0
[[145   0]
 [  0  55]]
             precision    recall  f1-score   support

         ok       1.00      1.00      1.00       145
   troubled       1.00      1.00      1.00        55

avg / total       1.00      1.00      1.00       200

94 Train accuracy: 1.0 Test accuracy: 1.0
[[154   0]
 [  0  46]]
             precision    recall  f1-score   support

         ok       1.00      1.00      1.00       154
   troubled       1.00      1.00      1.00        46

avg / total       1.00      1.00      1.00       200



95 Train accuracy: 1.0 Test accuracy: 1.0
[[164   0]
 [  0  36]]
             precision    recall  f1-score   support

         ok       1.00      1.00      1.00       164
   troubled       1.00      1.00      1.00        36

avg / total       1.00      1.00      1.00       200

96 Train accuracy: 1.0 Test accuracy: 1.0
[[154   0]
 [  0  46]]
             precision    recall  f1-score   support

         ok       1.00      1.00      1.00       154
   troubled       1.00      1.00      1.00        46

avg / total       1.00      1.00      1.00       200



97 Train accuracy: 1.0 Test accuracy: 1.0
[[161   0]
 [  0  39]]
             precision    recall  f1-score   support

         ok       1.00      1.00      1.00       161
   troubled       1.00      1.00      1.00        39

avg / total       1.00      1.00      1.00       200

98 Train accuracy: 1.0 Test accuracy: 1.0
[[155   0]
 [  0  45]]
             precision    recall  f1-score   support

         ok       1.00      1.00      1.00       155
   troubled       1.00      1.00      1.00        45

avg / total       1.00      1.00      1.00       200



99 Train accuracy: 1.0 Test accuracy: 1.0
[[150   0]
 [  0  50]]
             precision    recall  f1-score   support

         ok       1.00      1.00      1.00       150
   troubled       1.00      1.00      1.00        50

avg / total       1.00      1.00      1.00       200

